In [1]:
import pandas as pd
import numpy as np

geo to zip

In [29]:
tozipDataTemp = pd.read_csv('rowData\\zip_code_database.csv',encoding='latin1')
tozipDataTemp = tozipDataTemp.loc[tozipDataTemp['state']=='NY']
tozipDataTemp.columns
tozipDataTemp = tozipDataTemp.drop(['type', 'decommissioned', 'primary_city', 'acceptable_cities',\
       'unacceptable_cities', 'state', 'county', 'timezone', 'area_codes','world_region', 'country',\
       'irs_estimated_population_2014'],axis=1)
#tozipDataTemp.to_csv('data\\zip_code_database_clean.csv')

In [64]:
# return [zipcode, 4-neighbor zip area]
# find neariest zip-code by zip-center long/lat database
def trans_geo2zip(lat, long, retLen = 5, searchTreash=0.2):
    tozipData = pd.read_csv('data\\zip_code_database_clean.csv')
    #subData = tozipData
    subData = tozipData.loc[tozipData['longitude']>=long-searchTreash]
    subData = tozipData.loc[tozipData['longitude']<=long+searchTreash]
    subData = tozipData.loc[tozipData['latitude']>=lat-searchTreash]
    subData = tozipData.loc[tozipData['latitude']<=lat+searchTreash]
    dis=[]
    for index, row in subData.iterrows():
        distance = (row['longitude']-long)**2+(row['latitude']-lat)**2
        dis.append([int(row['zip']),distance])
    dis.sort(key=lambda x:x[1])
    #
    ret = dis[:retLen]
    return ret;
trans_geo2zip(41.11,-74.08)

[[10952, 0.0],
 [10982, 0.00039999999999984086],
 [10977, 0.0010000000000000284],
 [10901, 0.0036000000000002727],
 [10954, 0.0052999999999988855]]

Poplation - zip

In [2]:
populationDataDir = 'rowData\population_by_zip_2010.csv'
data = pd.read_csv(populationDataDir)
#data.head(2)

In [3]:
popuData = data.loc[data['gender'].isnull()]
popuData = popuData.drop(['minimum_age','maximum_age','gender'], axis=1)
#popuData.head(2)

In [4]:
zipData = pd.read_csv('rowData\zip_code_database.csv', encoding='latin1')
#zipData.head(2)

In [5]:
zipNYData = zipData.loc[zipData['state']=='NY'].zip.tolist()

In [6]:
popu = popuData.loc[popuData['zipcode'].isin(zipNYData)]
outFileDir = 'data\popu_zip.csv'
###############popu.to_csv(outFileDir)

Store - zip
three features: total, #individual, #business

In [7]:
storeRowData = pd.read_csv('rowData\Legally_Operating_Businesses.csv')
storeRowData.columns

Index(['DCA License Number', 'License Type', 'License Expiration Date',
       'License Category', 'Business Name', 'Business Name 2',
       'Address Building', 'Address Street Name',
       'Secondary Address Street Name', 'Address City', 'Address State',
       'Address ZIP', 'Contact Phone Number', 'Address Borough', 'Detail',
       'Longitude', 'Latitude'],
      dtype='object')

In [8]:
#NaN
storeRowData = storeRowData.fillna(0)
storeRowData = storeRowData[storeRowData['Address ZIP'].astype('str').apply(lambda x : x.isnumeric())]
#NYC city
cityList = ['NEW YORK','BROOKLYN','QUEENS VILLAGE','JAMAICA','FLUSHING','BRONX','COLLEGE POINT']

storesData = storeRowData[['Address ZIP','Address City','DCA License Number','License Type','License Expiration Date']]\
            .loc[storeRowData['Address ZIP'].astype('int64')<12000]
storesData = storesData.loc[storesData['Address ZIP'].astype('int64')>10000]
#print(len(storesData.index))
#storesData.head(3)
outFileDir = 'data\\NYC_Store_clean.csv'
##############3#storesData.to_csv(outFileDir)
storesData.isnull().sum()

Address ZIP                0
Address City               0
DCA License Number         0
License Type               0
License Expiration Date    0
dtype: int64

In [21]:
#describ stores
#storesData.groupby('Address ZIP').count().head(2)
#############storesData.groupby(['Address ZIP','License Type']).count().drop(['Address City','License Expiration Date'],axis=1)\
#############                .to_csv('data\\NYC_Store_Count.csv')

University/College - long/lat - zip
Museum - zip

In [70]:
univData = pd.read_csv('rowData\\COLLEGE_UNIVERSITY.csv', encoding='latin1')
univCount = storesData.groupby(['Address ZIP']).count().drop(['Address City','License Expiration Date','License Type'],axis=1)
univCount['count'] = 0
univCount=univCount.drop(['DCA License Number'],axis=1)

In [72]:
#univCount.loc['10011']
univRange = 5
for index, row in univData.iterrows():
    #univCount.loc[str(row.ZIP)] = univCount.loc[str(row.ZIP)]+1
    geomstr = row['the_geom']
    geom = geomstr.split()
    long = geom[1][1:]
    lat = geom[2][:-1]
    zips = trans_geo2zip(float(lat),float(long),univRange)
    pior = univRange
    for zip_dis in zips:
        zipIdx = str(zip_dis[0])
        #check index
        if zipIdx in univCount.index:
            univCount.loc[zipIdx] =  univCount.loc[zipIdx] + pior
        else:
            univCount.loc[zipIdx] = 0
        pior = pior - 1
#univCount.to_csv('data\\NYC_Univ_zip_count.csv')

In [75]:
musData = pd.read_csv('rowData\\MUSEUM.csv', encoding='latin1')
musCount = univCount.drop(['count'],axis=1)
musCount['count']=0
musRange = 5
for index, row in musData.iterrows():
    geomstr = row['the_geom']
    geom = geomstr.split()
    long = geom[1][1:]
    lat = geom[2][:-1]
    zips = trans_geo2zip(float(lat),float(long),musRange)
    pior = musRange
    for zip_dis in zips:
        zipIdx = str(zip_dis[0])
        #check index
        if zipIdx in musCount.index:
            musCount.loc[zipIdx] =  musCount.loc[zipIdx] + pior
        else:
            musCount.loc[zipIdx] = 0
        pior = pior - 1
#musCount.to_csv('data\\NYC_Mus_zip_count.csv')

In [76]:
libData = pd.read_csv('rowData\\LIBRARY.csv', encoding='latin1')
libCount = univCount.drop(['count'],axis=1)
libCount['count']=0
libRange = 3
for index, row in libData.iterrows():
    geomstr = row['the_geom']
    geom = geomstr.split()
    long = geom[1][1:]
    lat = geom[2][:-1]
    zips = trans_geo2zip(float(lat),float(long),libRange)
    pior = libRange
    for zip_dis in zips:
        zipIdx = str(zip_dis[0])
        #check index
        if zipIdx in libCount.index:
            libCount.loc[zipIdx] =  libCount.loc[zipIdx] + pior
        else:
            libCount.loc[zipIdx] = 0
        pior = pior - 1
#musCount.to_csv('data\\NYC_Lib_zip_count.csv')

Empty DataFrame
Columns: []
Index: [10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10010, 10011, 10012, 10013, 10014, 10016, 10017, 10018, 10019, 10020, 10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038, 10039, 10040, 10041, 10044, 10055, 10065, 10069, 10075, 10080, 10101, 10103, 10105, 10106, 10107, 10108, 10112, 10116, 10118, 10119, 10120, 10121, 10122, 10123, 10128, 10150, 10151, 10152, 10153, 10154, 10155, 10156, 10158, 10159, 10162, 10163, 10165, 10166, 10167, 10168, 10169, 10170, 10171, 10172, 10173, 10174, 10175, 10176, 10178, 10185, 10271, 10276, 10279, 10280, 10281, 10282, 10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, ...]

[557 rows x 0 columns]
